<img src="../../images/QISKit-c.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="250 px" align="left">

## _*Getting Started with QISKit SDK*_ 

For more information about how to use the Quantum Experience, consult the [Quantum Experience tutorials](https://quantumexperience.ng.bluemix.net/qstage/#/tutorial?sectionId=c59b3710b928891a1420190148a72cce&pageIndex=0) or check out QX [community forum](https://quantumexperience.ng.bluemix.net/qstage/#/community).

***
### Contributors
Ismael Faro, Jay M. Gambetta

## Quantum QISKit SDK tutorial

This tutorial aims to explain how to use the QISKit SDK, but from a developer's point-of-view. We review the steps it takes to install and start to use the SDK tools.

QISKIt is a Python software development kit (SDK) that you can use to create your quantum computing programs, based on circuits defined through the [OpenQASM 2.0 specification](https://github.com/IBM/qiskit-openqasm), compiled and then executed on several Backends (Real Quantum Processors online, Simulators online and Simulators on local). For the online Backends QISKit uses our [python API connector](https://github.com/IBM/qiskit-api-py) to the [IBM Quantum Experience project](http://quantumexperience.ng.bluemix.net/).

Once you get through this, we have other tutorials that introduce you to more complex concepts directly related with quantum computing.

More examples:
- Get a basic familiarity with the important concepts of [superposition and entanglement](superposition_and_entanglement.ipynb).
- Go beyond and explore a bit more in-depth in [superposition revisited](entanglement_revisited.ipynb).



## Install QISKit

The easiest way to install QISKit is by using the Anaconda python distribution.

- Install Anaconda: https://www.continuum.io/downloads

After that, install QISKit from the git repository

- Clone the repo:

```sh
git clone https://github.ibm.com/IBMQuantum/qiskit-sdk-py-dev
cd qiskit-sdk-py-dev
```

- Create the environment with the dependencies:

```sh
make env
```

## Use QISKit Python SDK

You can use the examples in an easy way with Jupyter or Python

Add your personal API token to the file "Qconfig.py" (get it from [IBM Quantum Experience](https://quantumexperience.ng.bluemix.net) > Account):

```sh
cp tutorial/Qconfig.py.default Qconfig.py
```
Run Jupyter notebook.

```sh
make run
```

## Basic Concept

### Create a Program

First you need to import the QuantumProgram package from QISKit

In [14]:
import sys
sys.path.append("../../") # solve the relative dependencies if you clone QISKit from the Git repo and use like a global.

from qiskit import QuantumProgram
import Qconfig

The basic elements that you need to create your first program are, the QuantumProgram,  a Circuit, a Quantum Register and a Classical Register.

In [16]:
# Creating Programs
# create your first QuantumProgram object instance.
Q_program = QuantumProgram()

# Creating Registers
# create your first Quantum Register called "qr" with 2 qubits 
qr = Q_program.create_quantum_registers("qr", 2)
# create your first Classical Register  called "cr" with 2 bits
cr = Q_program.create_classical_registers("cr", 2)

# Creating Circuits
# create your first Quantum Circuit called "qc" related to your Quantum Register "qr"
# and your Classical Register "cr"
qc = Q_program.create_circuit("qc", ["qr"], ["cr"])

>> quantum_registers created: qr 2
>> classical_registers created: cr 2


Another option to create your QuantumProgram instance is by defining a Dictionary with all the necessary components of your program.

In [17]:
Q_SPECS = {
    "name": "Program-tutorial",
    "circuits": [{
        "name": "Circuit",
        "quantum_registers": [{
            "name":"qr",
            "size": 4
        }],
        "classical_registers": [{
            "name":"cr",
            "size": 4
        }]}],
}

The required elements for a Program are a "name", and a "circuits" array; within "circuits" the required fields are "name" and it can have several Quantum registers and several Classical Registers. Every register needs to have a name and the number of the elements (qubits or bits).

After that, you can use this dictionary definition as the specs of one QuantumProgram object to initialize it.

In [18]:
new_Q_program = QuantumProgram(specs=Q_SPECS)

>> quantum_registers created: qr 4
>> classical_registers created: cr 4


You can also get every component from your new_Q_program to use.

In [19]:
#get the components.

#get the circuit by Name
circuit = new_Q_program.circuit("Circuit")

#get the Quantum Register by Name
quantum_r = new_Q_program.quantum_registers("qr")

#get the Quantum Register by Name
classical_r = new_Q_program.classical_registers('cr')

### Add Gates to your Circuit
After you create the circuit with its registers you can add gates to manipulate the registers. The next is the example of the all the gates that you can use.

You can find extensive information about these gates and how use it into our [Quantum Experience User Guide](https://quantumexperience.ng.bluemix.net/qstage/#/tutorial?sectionId=71972f437b08e12d1f465a8857f4514c&pageIndex=2) 

In [21]:
# H (Hadamard) gate to the Qbit 0 in the Quantum Register "qr" 
circuit.h(quantum_r[0])

# Pauli X gate to the Qbit 1 in the Quantum Register "qr" 
circuit.x(quantum_r[1])

# Pauli Y gate to the Qbit 2 in the Quantum Register "qr" 
circuit.y(quantum_r[2])

# Pauli Z gate to the Qbit 3 in the Quantum Register "qr" 
circuit.z(quantum_r[3])

# Cnot (Controlled-NOT)gate from Qbit 0 to the Qbit 2
circuit.cx(quantum_r[0], quantum_r[2])

# add a barrier to your circuit
circuit.barrier()

# first physical gate: u1(lambda) to Qbit 0
circuit.u1(0.3, quantum_r[0])

# second physical gate: u2(phi,lambda) to Qbit 1
circuit.u2(0.3, 0.2, quantum_r[1])

# second physical gate: u3(theta,phi,lambda) to Qbit 2
circuit.u3(0.3, 0.2, 0.1, quantum_r[2])

# S Phase gate to Qbit 0
circuit.s(quantum_r[0])

# T Phase gate to Qbit 1
circuit.t(quantum_r[1])

# identity gate to Qbit 1
circuit.iden(quantum_r[1])

# Classical if, from Qbit2 gate Z to classical bit 1
# circuit.z(quantum_r[2]).c_if(classical_r, 1)

# measure gate from the Qbit 0 to Classical bit 0
circuit.measure(quantum_r[0], classical_r[0])


### extract QASM

You can obtain a QASM representation of your code.

In [22]:
# QASM from a circuit

QASM_source = circuit.qasm()

print (QASM_source)

OPENQASM 2.0;
include "qelib1.inc";
qreg qr[4];
creg cr[4];
h qr[0];
x qr[1];
y qr[2];
z qr[3];
cx qr[0],qr[2];
barrier qr[0],qr[1],qr[2],qr[3];
u1(0.300000000000000) qr[0];
u2(0.300000000000000,0.200000000000000) qr[1];
u3(0.300000000000000,0.200000000000000,0.100000000000000) qr[2];
s qr[0];
t qr[1];
id qr[1];
measure qr[0] -> cr[0];
h qr[0];
x qr[1];
y qr[2];
z qr[3];
cx qr[0],qr[2];
barrier qr[0],qr[1],qr[2],qr[3];
u1(0.300000000000000) qr[0];
u2(0.300000000000000,0.200000000000000) qr[1];
u3(0.300000000000000,0.200000000000000,0.100000000000000) qr[2];
s qr[0];
t qr[1];
id qr[1];
measure qr[0] -> cr[0];



In [23]:
# Qasm from a program

QASM_source = new_Q_program.program_to_text()

print(QASM_source)

# Circuit: Circuit
OPENQASM 2.0;
include "qelib1.inc";
qreg qr[4];
creg cr[4];
u1(3.141592653589793) qr[3];
u3(3.141592653589793,1.5707963267948966,1.5707963267948966) qr[2];
u3(3.141592653589793,0.0,3.141592653589793) qr[1];
u2(0.0,3.141592653589793) qr[0];
cx qr[0],qr[2];
barrier qr[0],qr[1],qr[2],qr[3];
u1(0.3) qr[0];
u1(1.5707963267948966) qr[0];
measure qr[0] -> cr[0];
u2(0.0,3.141592653589793) qr[0];
u2(0.3,0.2) qr[1];
u1(0.7853981633974483) qr[1];
id qr[1];
u3(3.141592653589793,0.0,3.141592653589793) qr[1];
u3(0.3,0.2,0.1) qr[2];
u3(3.141592653589793,1.5707963267948966,1.5707963267948966) qr[2];
cx qr[0],qr[2];
u1(3.141592653589793) qr[3];
barrier qr[0],qr[1],qr[2],qr[3];
u1(0.3) qr[0];
u1(1.5707963267948966) qr[0];
measure qr[0] -> cr[0];
u2(0.3,0.2) qr[1];
u1(0.7853981633974483) qr[1];
id qr[1];
u3(0.3,0.2,0.1) qr[2];


Note: the above two codes do not have the same notation. The second one returns a conversion where every single qubit gate {x, y, z, h, s, t} is replaced with the arbitrary u1, u2 and u3 gate representation.

### Compile & Run or Execute

In [24]:
device = 'simulator' #Backend where to execute your program, in this case it is the online simulator
circuits = [circuit]  #Group of circuits to execute

Q_program.set_api(Qconfig.APItoken, Qconfig.config["url"]) #set the APIToken and API url

True

In [26]:
Q_program.compile(circuits, device) #Compile your program

result = Q_program.run(wait=2, timeout=240) #Run your program in the device and check the execution result every 2 seconds 

print(result)

backend that is running simulator
status = RUNNING (2 seconds)
status = RUNNING (4 seconds)
status = RUNNING (6 seconds)
{'backend': {'name': 'simulator'}, 'compiled_circuits': [{'qasm': '\ninclude "qelib1.inc";\nqreg qr[4];\ncreg cr[4];\nu1(3.141592653589793) qr[3];\nu3(3.141592653589793,1.5707963267948966,1.5707963267948966) qr[2];\nu3(3.141592653589793,0.0,3.141592653589793) qr[1];\nu2(0.0,3.141592653589793) qr[0];\ncx qr[0],qr[2];\nbarrier qr[0],qr[1],qr[2],qr[3];\nu1(0.3) qr[0];\nu1(1.5707963267948966) qr[0];\nmeasure qr[0] -> cr[0];\nu2(0.0,3.141592653589793) qr[0];\nu2(0.3,0.2) qr[1];\nu1(0.7853981633974483) qr[1];\nid qr[1];\nu3(3.141592653589793,0.0,3.141592653589793) qr[1];\nu3(0.3,0.2,0.1) qr[2];\nu3(3.141592653589793,1.5707963267948966,1.5707963267948966) qr[2];\ncx qr[0],qr[2];\nu1(3.141592653589793) qr[3];\nbarrier qr[0],qr[1],qr[2],qr[3];\nu1(0.3) qr[0];\nu1(1.5707963267948966) qr[0];\nmeasure qr[0] -> cr[0];\nu2(0.3,0.2) qr[1];\nu1(0.7853981633974483) qr[1];\nid qr[1];\

When you run a program the possible results will be:

```
JOB_STATUS = {
    inProgress: 'RUNNING',
    errorOnCreate: 'ERROR_CREATING_JOB',
    errorExecuting: 'ERROR_RUNNING_JOB',
    completed: 'COMPLETED'
  };
```

The *run()* command waiting until timeout or when receive the "COMPLETED" or some error message.

You can use the *execute()* to combine the compile and run in a single step.

In [27]:
result = Q_program.execute(circuits, device, wait=2, timeout=240)

backend that is running simulator
status = RUNNING (2 seconds)
status = RUNNING (4 seconds)
status = RUNNING (6 seconds)


#### Compile parameters
Q_program.compile(circuits, device="simulator", shots=1024, max_credits=3, basis_gates=None, coupling_map=None)
     - circuits: Array of circuit to compile
     - device: Backend 
         ["ibmqx_qasm_simulator",   # Online simulator
         "ibmqx2",                  # Online RealChip, 5Qbits
         "ibmqx3",                  # Online RealChip, 16Qbits
         "local_unitary_simulator", # Local unitary Simulator 
         "local_qasm_simulator"]    # Local Simulator 
     - shots: Number of shots, only for real chips and qasm simulators
     - max_credits: Maximum number of the credits to spend in the executions. If the executions are more expensives, the job is aborted, only the real chips
     - basis_gates: are the base gates, by default are: u1,u2,u3,cx,id
     - coupling_map: Object that represent the physical/topological Layout of a chip.
#### Run parameters
Q_program.run(wait=5, timeout=60)
     - wait: Time to wait to check if the execution is COMPLETED.
     - timeout: Timeout of the execution.
#### Execute parameters 
*the Execute has the combined parameters of Compile and Run*

Q_program.execute(circuits, device, shots=1024,
                max_credits=3, basis_gates=None, wait=5, timeout=60, basis_gates=None, coupling_map=None,)

### Execute on a real device

In [12]:
device = 'IBMQX5qv2'   #Backend where execute your program, in this case in the Real Quantum Chip online 
circuits = [circuit]   #Group of circuits to exec
shots = 1024           #Number of Shots to run the program (experiment), Maximum 8192 shots.
max_credits=3          #Maximum number of the credits to spend in the executions. 

result = Q_program.execute(circuits, device, shots, max_credits=3, wait=10, timeout=240)

backend that is running IBMQX5qv2
status = RUNNING (10 seconds)
status = RUNNING (20 seconds)
status = RUNNING (30 seconds)
status = RUNNING (40 seconds)
status = RUNNING (50 seconds)
status = RUNNING (60 seconds)
status = RUNNING (70 seconds)
status = RUNNING (80 seconds)
status = RUNNING (90 seconds)
status = RUNNING (100 seconds)
status = RUNNING (110 seconds)
status = RUNNING (120 seconds)
status = RUNNING (130 seconds)
status = RUNNING (140 seconds)
status = RUNNING (150 seconds)
status = RUNNING (160 seconds)
status = RUNNING (170 seconds)
status = RUNNING (180 seconds)
status = RUNNING (190 seconds)
status = RUNNING (200 seconds)
status = RUNNING (210 seconds)
status = RUNNING (220 seconds)
status = RUNNING (230 seconds)
status = RUNNING (240 seconds)


### Result
You can access to the result via the element: qasms[n].result.data.counts 

In [13]:
result['qasms'][0]['result']['data']['counts']

KeyError: 'qasms'

or you can use the function tool get_counts(n) to obtain it directly

In [ ]:
Q_program.get_counts(0) 